<a href="https://colab.research.google.com/github/AnikaD15/EN-HU-Translation/blob/main/milestone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
## GET DATA 
!!curl -O http://www.manythings.org/anki/hun-eng.zip
!!unzip hun-eng.zip

['Archive:  hun-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ']

In [23]:
## Config 
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "hun.txt"

## Prepare Data

In [24]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    # input_text, target_text, _ = line.split("\t")
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 81
Number of unique output tokens: 72
Max sequence length for inputs: 54
Max sequence length for outputs: 17


In [25]:
# character's lexographical order
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [26]:
# why is this neded?
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [27]:
# huh
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [28]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

## Define Model

In [29]:
# Define the model that will turn 
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Train Model

In [30]:
# training, validation, test data
valid_split = 0.2
test_split  = 0.1

# array index boundary
v_point = int(encoder_input_data.shape[0]*(1-valid_split-test_split))
t_point = int(encoder_input_data.shape[0]*(1-test_split))

encoder_input_data_train = encoder_input_data[:t_point]
decoder_input_data_train = decoder_input_data[:t_point]
decoder_target_data_train = decoder_target_data[:t_point]

encoder_input_data_valid = encoder_input_data[v_point:t_point]
decoder_input_data_valid = decoder_input_data[v_point:t_point]
decoder_target_data_valid = decoder_target_data[v_point:t_point]

encoder_input_data_test = encoder_input_data[t_point:]
decoder_input_data_test = decoder_input_data[t_point:]
decoder_target_data_test = decoder_target_data[t_point:]

In [31]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [32]:
es = EarlyStopping(monitor='val_accuracy', 
                   patience=20, verbose=1,
                   restore_best_weights=True)

In [ ]:
model.fit(
    [encoder_input_data_train, decoder_input_data_train],
    decoder_target_data_train,
    batch_size=batch_size,
    callbacks=[es],
    epochs=epochs,
    validation_data = ([encoder_input_data_valid, decoder_input_data_valid],
    decoder_target_data_valid)
)
# Save model
model.save("hu-en_model")

Epoch 1/100
141/141 [==============================] - 61s 399ms/step - loss: 2.4199 - accuracy: 0.3582 - val_loss: 2.3470 - val_accuracy: 0.3389
Epoch 2/100
141/141 [==============================] - 54s 380ms/step - loss: 1.8554 - accuracy: 0.4898 - val_loss: 1.9713 - val_accuracy: 0.4444
Epoch 3/100
141/141 [==============================] - 54s 386ms/step - loss: 1.5863 - accuracy: 0.5413 - val_loss: 1.7419 - val_accuracy: 0.4878
Epoch 4/100
141/141 [==============================] - 56s 400ms/step - loss: 1.4974 - accuracy: 0.5650 - val_loss: 1.6450 - val_accuracy: 0.5216
Epoch 5/100
141/141 [==============================] - 54s 384ms/step - loss: 1.3848 - accuracy: 0.5903 - val_loss: 1.5999 - val_accuracy: 0.5195
Epoch 6/100
141/141 [==============================] - 56s 394ms/step - loss: 1.3150 - accuracy: 0.6092 - val_loss: 1.4926 - val_accuracy: 0.5537
Epoch 7/100
141/141 [==============================] - 53s 372ms/step - loss: 1.2556 - accuracy: 0.6252 - val_loss: 1.4293 -

## Evaluate Model

In [ ]:
model.evaluate([encoder_input_data_test, decoder_input_data_test],
    decoder_target_data_test)

## Decode

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("hu-en_model")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)